# Gower distance 기반 군집 내 최유사 유저

In [1]:
import pandas as pd
import numpy as np
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler
from pathlib import Path

BASE_DIR = Path().resolve().parent.parent.parent
base_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "1.여행로그 장소 추천 고도화" 
data_path = base_path / "data"
save_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "3.여행루트추천" / "model"
gps_data_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "3.여행루트추천" / "data"

df_raw = pd.read_csv(save_path / "rating_user_info_all_cluster.csv")

In [2]:
from joblib import load

model_path = save_path / "user_cluster_kproto.joblib"
artifacts = load(model_path)

kproto = artifacts["kproto"]
scaler = artifacts["scaler"]
cat_cols = artifacts["cat_cols"]
num_cols = artifacts["num_cols"]
cat_idx = artifacts["cat_idx"]
cluster_labels = artifacts["cluster_labels"]
num_medians = artifacts["num_medians"]
cluster_name_map = artifacts["cluster_labels_name_map"]

print("✅ 모델 아티팩트 로드 완료")

✅ 모델 아티팩트 로드 완료


In [3]:
# 수치형 컬럼별 min/max
num_mins = df_raw[num_cols].min()
num_maxs = df_raw[num_cols].max()
num_ranges = num_maxs - num_mins
# 0으로 나누는 것 방지
num_ranges[num_ranges == 0] = 1.0


In [4]:
# ---------------------------------------------------------
# 신규 유저용 raw 전처리 함수
# ---------------------------------------------------------
def preprocess_new_user_raw(user_dict):
    new_df = pd.DataFrame([user_dict])

    # 없는 컬럼 자동 보정
    for col in cat_cols:
        if col not in new_df.columns:
            new_df[col] = "Unknown"
    for col in num_cols:
        if col not in new_df.columns:
            new_df[col] = np.nan

    # 결측치 처리
    new_df[cat_cols] = new_df[cat_cols].fillna("Unknown")
    for c in num_cols:
        new_df[c] = new_df[c].fillna(num_medians[c])

    # 타입 정리
    new_df[cat_cols] = new_df[cat_cols].astype(str)
    # 수치형은 float로 두면 됨
    for c in num_cols:
        new_df[c] = new_df[c].astype(float)

    return new_df


In [5]:
# ---------------------------------------------------------
# 군집 내 Gower distance로 최유사 유저 찾기
# ---------------------------------------------------------
def find_most_similar_in_cluster_gower(new_user_raw_df, cluster_id, topn=3):
    """
    new_user_raw_df: preprocess_new_user_raw(user_dict) 결과 (1-row DF, raw scale)
    cluster_id: 이미 예측된 군집 번호

    return: (가장 비슷한 기존 유저 row (df_raw에서), Gower distance 값)
    """
    # 이 군집에 속한 기존 유저만 선택 (raw 기준)
    cluster_members_raw = df_raw[df_raw["cluster"] == cluster_id].copy()

    if len(cluster_members_raw) == 0:
        return None, None

    # 수치형 Gower 부분: |x - y| / range
    # new_user_raw_df[num_cols].iloc[0] : 1D Series
    num_diff = (
        (cluster_members_raw[num_cols].astype(float)
         - new_user_raw_df[num_cols].iloc[0].astype(float))
        .abs()
        / num_ranges
    )

    # 범주형 Gower 부분: 같으면 0, 다르면 1
    cat_diff = (
        cluster_members_raw[cat_cols].astype(str)
        != new_user_raw_df[cat_cols].iloc[0].astype(str)
    ).astype(int)

    # feature-wise distance를 모두 합쳐서 평균
    all_diffs = pd.concat([num_diff, cat_diff], axis=1)
    gower_dist = all_diffs.mean(axis=1)  # 각 row별 Gower distance

    # best_idx = gower_dist.idxmin()
    # best_user = cluster_members_raw.loc[best_idx]
    # best_dist = float(gower_dist.loc[best_idx])

    # return best_user, best_dist

    topn_idx = gower_dist.nsmallest(topn).index
    topn_users = cluster_members_raw.loc[topn_idx].reset_index(drop=True).copy()
    topn_dists = gower_dist.loc[topn_idx].astype(float).tolist()

    return topn_users, topn_dists


In [6]:
def assign_new_user(user_dict):
    """
    완전 신규 유저에 대해 군집 ID와 라벨을 반환하는 함수.

    user_dict 예시:
    {
        "GENDER": "남",
        "AGE_GRP": "30",
        "MARR_STTS": "1",
        "JOB_NM": "3",
        "INCOME": 4,
        "TRAVEL_STYL_1": "2",
        "TRAVEL_STATUS_RESIDENCE": "서울특별시",
        "TRAVEL_STATUS_ACCOMPANY": "2인 여행(가족 외)",
        "TRAVEL_MOTIVE_1": "7",
        "TRAVEL_NUM": 3,
        "TRAVEL_COMPANIONS_NUM": 1,
        "MONTH": "8",
        "SEASON": "summer",
        "HOW_LONG": 3,
    }
    """
    # 1) dict → DataFrame
    new_df = pd.DataFrame([user_dict])

    # 2) 없는 컬럼 자동 추가 (완전 신규 유저 대비)
    for col in cat_cols:
        if col not in new_df.columns:
            new_df[col] = "Unknown"
    for col in num_cols:
        if col not in new_df.columns:
            new_df[col] = np.nan

    # 3) 결측값 처리
    new_df[cat_cols] = new_df[cat_cols].fillna("Unknown")
    for c in num_cols:
        new_df[c] = new_df[c].fillna(num_medians[c])

    # 4) 타입 정리
    new_df[cat_cols] = new_df[cat_cols].astype(str)

    # 5) 스케일링
    new_df[num_cols] = scaler.transform(new_df[num_cols])

    # 6) numpy 변환 (dtype=object 필수)
    new_np = new_df[cat_cols + num_cols].to_numpy(dtype=object)

    # 7) 군집 예측
    cluster = int(kproto.predict(new_np, categorical=cat_idx)[0])
    label = cluster_labels[cluster]

    return cluster, label


In [7]:
def assign_and_find_similar_gower(user_dict):
    """
    1) 신규 유저 군집 예측
    2) 그 군집 안에서 Gower distance 기준 최유사 유저 찾기
    """
    # 1) 군집 예측 (기존에 만든 함수 사용)
    cluster_id, label = assign_new_user(user_dict)

    # 2) raw 전처리
    new_user_raw_df = preprocess_new_user_raw(user_dict)

    # 3) Gower 기반 최유사 유저 탐색
    topn_user, topn_dist = find_most_similar_in_cluster_gower(new_user_raw_df, cluster_id)

    return {
        "cluster_id": cluster_id,
        "cluster_label": label,
        "cluster_name": cluster_name_map[cluster_id],
        "similar_user": topn_user,  # df_raw의 한 row (Series)
        "distance": topn_dist,
    }


In [8]:
example_user = {
    "GENDER": "남",
    "AGE_GRP": "30",
    "MARR_STTS": "1",
    "JOB_NM": "3",
    "INCOME": 4,
    "TRAVEL_STYL_1": "2",
    "TRAVEL_STATUS_RESIDENCE": "서울특별자치도",
    "TRAVEL_STATUS_ACCOMPANY": "2인 여행(가족 외)",
    "TRAVEL_MOTIVE_1": "7",
    "TRAVEL_NUM": 3,
    "TRAVEL_COMPANIONS_NUM": 1,
    "MONTH": "8",
    "SEASON": "summer",
    "HOW_LONG": 3,
}

result = assign_and_find_similar_gower(example_user)

print("신규 유저 군집 ID:", result["cluster_id"])
print("신규 유저 군집 라벨:", result["cluster_label"])
print("신규 유저 군집 라벨 이름:", result["cluster_name"])
print("\n가장 비슷한 기존 유저:")
if result["similar_user"] is not None:
    cols_to_show = ["TRAVELER_ID"] + cat_cols + num_cols if "TRAVELER_ID" in result["similar_user"].index else cat_cols + num_cols
    
    for i in range(len(result["similar_user"])):
        print(result["similar_user"].iloc[i][cols_to_show])
        print("\nGower distance:", result["distance"][i])
else:
    print("해당 군집에 기존 유저가 없습니다.")


신규 유저 군집 ID: 4
신규 유저 군집 라벨: 30대 · 2인 여행(가족 외) · 자연 중간선호 · 평균 3.6일 여행자
신규 유저 군집 라벨 이름: 느슨하게 떠나는 듀오 무드러

가장 비슷한 기존 유저:
GENDER                               남
AGE_GRP                             30
MARR_STTS                            1
JOB_NM                               3
TRAVEL_STYL_1                        2
TRAVEL_STATUS_RESIDENCE          광주광역시
TRAVEL_STATUS_ACCOMPANY    2인 여행(가족 외)
TRAVEL_MOTIVE_1                      3
SEASON                          summer
MONTH                                8
TRAVEL_NUM                           1
TRAVEL_COMPANIONS_NUM                1
HOW_LONG                             3
INCOME                               4
Name: 0, dtype: object

Gower distance: 0.14778325123152708
GENDER                               남
AGE_GRP                             30
MARR_STTS                            1
JOB_NM                               3
TRAVEL_STYL_1                        2
TRAVEL_STATUS_RESIDENCE          부산광역시
TRAVEL_STATUS_ACCOMPANY    2인 여행(가족 외)
TRAV

In [9]:
topn_traveler_id = result["similar_user"].TRAVELER_ID.to_list()

In [10]:
print(topn_traveler_id)

['h004211', 'h002667', 'h005746']


In [11]:
for id in topn_traveler_id:
    fname = str(id) + ".csv"
    df_tmp = pd.read_csv(gps_data_path / fname)
    print(df_tmp)

   Unnamed: 0.1  Unnamed: 0  MOBILE_NUM_ID     X_COORD    Y_COORD  \
0             0           1  h004211_80839  126.329890  33.246089   
1             1           2  h004211_80839  126.299779  33.225816   
2             2           3  h004211_80839  126.675563  33.435051   
3             3           4  h004211_80839  126.804443  33.545862   
4             4           5  h004211_80839  126.843479  33.530529   
5             5           6  h004211_80839  126.842431  33.533294   
6             6           7  h004211_80839  126.795921  33.532067   
7             7           8  h004211_80839  126.665361  33.541894   
8             8           9  h004211_80839  126.377223  33.483825   
9             9          10  h004211_80839  126.312798  33.462531   

                DT_MIN  TRAVEL_ID  TRAVEL_DAY                  ADDRESS_FULL  \
0  2023-09-01 12:00:00  h_h004211           1  제주특별자치도 서귀포시 안덕면 화순리 1050-16   
1  2023-09-01 13:10:00  h_h004211           1     제주특별자치도 서귀포시 안덕면 형제해안로 70   
2  